# Demonstration of Gibbs Sampler on a forward Model with 4 Parameters

In [ ]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns
import pandas as pd

# Configuration
np.random.seed(42)
param_names = ["m0", "m1", "m2", "m3"]
model_string = "Model: y = m0 / (1 + exp(-m1*(x - m2))) + m3*sin(x)"
true_params = [3.0, 2.0, 5.0, 0.2]

def forward_model(x, m):
    return m[0] / (1 + np.exp(-m[1]*(x - m[2]))) + m[3] * np.sin(x)

x_data = np.linspace(0, 10, 800)
y_true = forward_model(x_data, true_params)

# Simulated observations
sigma_low = 0.02
sigma_high = 2.0

# Gibbs Sampling parameters
n_samples = 10000
burn_in = n_samples // 2

init_guess = [1.0, 1.0, 1.0, 1.0]

step_sizes = [0.5, 0.2, 0.2, 0.1]
step_sizes = [0.1, 0.1, 0.1, 0.1]


In [ ]:
def plot_data_model_fit(x, y_true, y_obs, mean_params, label):
    plt.figure(figsize=(10, 4))
    plt.plot(x, y_true, 'k-', label='True')
    plt.plot(x, y_obs, '.', label='Observed', alpha=0.4)
    plt.plot(x, forward_model(x, mean_params), 'r--', label='Inferred')
    plt.title(f'{label} Fit')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def triangle_plot(data, param_names, true_params, inferred_params, label, half_range=0.5):
    """
    Create a triangle plot showing posterior distributions.
    - data: MCMC samples (numpy array or ArviZ InferenceData)
    - param_names: list of parameter names
    - true_params: list of true parameter values
    - inferred_params: list of inferred parameter values (mean or MAP)
    - label: title label for the plot
    - half_range: float, half-width around mean/true for axis limits (default = 0.5)
    """
    if isinstance(data, np.ndarray):
        df = pd.DataFrame(data, columns=param_names)
    else:
        df = az.extract(data, var_names=param_names).to_dataframe()[param_names]
    
    means = df.mean()

    g = sns.PairGrid(df, vars=param_names, corner=True)
    g.map_lower(sns.kdeplot, levels=5, fill=False)
    g.map_diag(sns.kdeplot, fill=True)

    # Calculate dynamic axis limits
    lower_limits = []
    upper_limits = []
    for i, p in enumerate(param_names):
        mean_val = means[p]
        true_val = true_params[i]
        lower = min(mean_val, true_val) - half_range
        upper = max(mean_val, true_val) + half_range
        lower_limits.append(lower)
        upper_limits.append(upper)

    # Set axis limits for marginals and KDEs
    for i, p in enumerate(param_names):
        ax = g.axes[i, i]
        ax.axvline(true_params[i], color='k', linestyle='-')
        ax.axvline(inferred_params[i], color='r', linestyle='--')
        ax.set_xlim(lower_limits[i], upper_limits[i])

    for i in range(len(param_names)):
        for j in range(i):
            ax = g.axes[i, j]
            ax.set_xlim(lower_limits[j], upper_limits[j])
            ax.set_ylim(lower_limits[i], upper_limits[i])

    plt.suptitle(label + f" - Triangle Plot (half-range = {half_range})", y=1.02)
    plt.show()

def print_summary(model_string, true_vals, inferred_vals, label):
    print(f"--- {label} Summary ---")
    print(f"{model_string}")

    print(f"{'Param':<5} {'True':>10} {'Inferred':>10} {'Error':>10}")
    for name, t, p in zip(param_names, true_vals, inferred_vals):
        print(f"{name:<5} {t:10.4f} {p:10.4f} {p - t:10.4f}")

In [ ]:
def log_likelihood(p, x, y_obs, sigma):
    y_model = forward_model(x, p)
    return -0.5 * np.sum(((y_obs - y_model) / sigma) ** 2)

def gibbs_sampler(x, y_obs, sigma, init_params, num_samples, step_size):
    samples = np.zeros((num_samples, len(init_params)))
    current = np.array(init_params)
    for i in range(num_samples):
        for j in range(len(init_params)):
            proposal = current.copy()
            proposal[j] += np.random.normal(0, step_size[j])
            if np.random.rand() < np.exp(log_likelihood(proposal, x, y_obs, sigma) - log_likelihood(current, x, y_obs, sigma)):
                current[j] = proposal[j]
        samples[i, :] = current
    return samples

## 🔵 Inference on Low Noise Data

In [ ]:
sigma = sigma_low
label = f"Low Noise: sigma={sigma_low}"
y_obs = y_true + np.random.normal(0, sigma, size=len(x_data))

samples = gibbs_sampler(x_data, y_obs, sigma, init_guess, n_samples, step_sizes)
posterior = samples[burn_in:]
mean = np.mean(posterior, axis=0)

print_summary(model_string, true_params, mean, label)
plot_data_model_fit(x_data, y_true, y_obs, mean, label)
triangle_plot(posterior, param_names, true_params, mean, label, half_range=0.2)

## 🔴 Inference on High Noise Data 

In [ ]:
sigma = sigma_high
label = f"High Noise: sigma={sigma}"

y_obs = y_true + np.random.normal(0, sigma, size=len(x_data))

samples = gibbs_sampler(x_data, y_obs, sigma, init_guess, n_samples, step_sizes)
posterior = samples[burn_in:]
mean = np.mean(posterior, axis=0)

print_summary(model_string, true_params, mean, label)
plot_data_model_fit(x_data, y_true, y_obs, mean, label)
triangle_plot(posterior, param_names, true_params, mean, label, half_range=0.5)